In [1]:
# example_meta_data_reader.ipynb
# Authors: Stephan Meighen-Berger
# Shows how to deal with the output files

In [2]:
import awkward as ak
import pyarrow.parquet as pq

In [3]:
example_set = ak.from_parquet('./output/custom_1337_meta_data.parquet')

In [4]:
example_set.fields

['event_id', 'mc_truth', 'lepton', 'hadron', 'total']

In [5]:
example_set.event_id

<Array [0, 1, 2, 3, 4] type='5 * int64'>

In [15]:
# The events are split into three parts:
#   - lepton: the light from the generated lepton
#   - hadron: the light produced from the associated hadronic cascade (at production point)
#   - total: the light from both parts
# Each of these fields is formatted the same way
example_set.total.fields

['sensor_id',
 'sensor_pos_x',
 'sensor_pos_y',
 'sensor_pos_z',
 'sensor_string_id',
 't']

In [21]:
# Example for one event
event_id = 4
# The sensors hit
print("Total number of hits:")
print(len(example_set.total.sensor_id[event_id]))
print('The hit sensors: ')
print(example_set.total.sensor_id[event_id])
print('Their associated strings')
print(example_set.total.sensor_string_id[event_id])
print('The hit times')
print(example_set.total.t[event_id])

Total number of hits:
1305
The hit sensors: 
[33, 71, 113, 147, 148, 256, 258, 259, ... 2265, 2325, 2504, 2505, 2506, 2506, 2506]
Their associated string
[1, 3, 5, 6, 6, 11, 11, 11, 13, 16, ... 92, 92, 102, 105, 113, 113, 113, 113, 113]
The hit time
[572, 681, 799, 353, 491, 370, 359, 263, ... 159, 583, 315, 210, 197, 188, 185, 188]
